In [19]:
import numpy as np
import gymnasium as gym
import random
from gymnasium import spaces
from collections import deque

In [20]:
def bfs_reachable(grid, start, targets):
    """
    Check if all target cells are reachable from start on grid (0=free, 1=obstacle).
    """
    H, W = grid.shape
    visited = np.zeros_like(grid, dtype=bool)
    queue = deque([start])
    visited[start] = True
    reached = set()
    while queue:
        i, j = queue.popleft()
        if (i, j) in targets:
            reached.add((i, j))
            if reached == set(targets):
                return True
        for di, dj in ((1,0),(-1,0),(0,1),(0,-1)):
            ni, nj = i+di, j+dj
            if 0 <= ni < H and 0 <= nj < W and not visited[ni, nj] and grid[ni, nj] == 0:
                visited[ni, nj] = True
                queue.append((ni, nj))
    return False


In [21]:
class CoverageEnv(gym.Env):
    metadata = {"render.modes": ["human"]}

    def __init__(self, max_steps=200, seed=None):
        super().__init__()
        self.H, self.W = 8, 8
        self.max_steps = max_steps

        # seeding for reproducibility
        self.seed(seed)

        # Action: down, up, right, left
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Box(0.0, 1.0, shape=(5, self.H, self.W), dtype=np.float32)

        # Define a fixed shape library
        self.shape_library = [
            np.array([[1]]),                # single cell
            np.ones((1,3), dtype=int),     # horizontal bar
            np.ones((2,2), dtype=int),     # 2x2 block
            np.array([[1,1,1],             # U-shape
                      [1,0,1],
                      [1,1,1]]),
            np.array([[1,1,0],             # L-shape
                      [1,0,0]])
        ]

    def seed(self, seed=None):
        """
        Seed the environment's RNGs for reproducible layouts.
        """
        np.random.seed(seed)
        random.seed(seed)
        return [seed]

    def reset(self, *, seed=None, options=None):
        """
        Reset the environment; returns obs (shape C×H×W), info
        Channels:
         0 = free space
         1 = obstacles
         2 = agent location
         3 = target area
         4 = visited mask (all zeros at reset)
        """

        if seed is not None:
            self.seed(42)
        else: 
            self.seed(42)


        while True:
            grid = np.zeros((self.H, self.W), dtype=int)

            num_shapes = np.random.randint(1, len(self.shape_library) + 1)

            allowed = random.sample(self.shape_library, num_shapes)

            placed = np.zeros_like(grid)
            for _ in range(num_shapes):
                shape = random.choice(allowed)
                sh, sw = shape.shape
                i = np.random.randint(0, self.H - sh + 1)
                j = np.random.randint(0, self.W - sw + 1)
                if not np.any(placed[i:i+sh, j:j+sw] & shape):
                    placed[i:i+sh, j:j+sw] |= shape
            grid = placed

            ti = np.random.randint(0, self.H - 3 + 1)
            tj = np.random.randint(0, self.W - 3 + 1)
            full_block = [(ti+di, tj+dj) for di in range(3) for dj in range(3)]
            targets = [(i,j) for (i,j) in full_block if grid[i,j] == 0]
            if not targets:
                continue

            free_cells = list(zip(*np.where(grid == 0)))
            start = random.choice(free_cells)
            if bfs_reachable(grid, start, targets):
                break

        self.grid = grid
        self.targets = set(targets)
        self.agent_pos = start
        self.visited = { start }
        self.steps = 0

        return self._get_obs(), {}

    def _get_obs(self):
        C = 5
        state = np.zeros((C, self.H, self.W), dtype=np.float32)

        # channel 0: free space (grid==0)
        state[0, :, :] = (self.grid == 0).astype(np.float32)
        # channel 1: obstacles (grid!=0)
        state[1, :, :] = (self.grid != 0).astype(np.float32)
        # channel 2: agent location
        i, j = self.agent_pos
        state[2, i, j] = 1.0
        # channel 3: target area
        for (ti, tj) in self.targets:
            state[3, ti, tj] = 1.0
        for vi, vj in self.visited:
            state[4, vi, vj] = 1.0
        
        return state

    def step(self, action):
        # Define movement vectors
        # 0 = down, 1 = up, 2 = right, 3 = left
        moves = {0: (1, 0), 1: (-1, 0), 2: (0, 1), 3: (0, -1)}
        i, j = self.agent_pos
        di, dj = moves[action]
        ni, nj = i + di, j + dj

        # Default baseline reward
        reward = 0

        # Check validity and apply penalties
        if not (0 <= ni < self.H and 0 <= nj < self.W and self.grid[ni, nj] == 0):
            # Invalid action: stay in place
            self.agent_pos = (i, j) 
        else:
            # Valid move: update position
            self.agent_pos = (ni, nj)

        # Check if on a target
        if self.agent_pos not in self.visited:
            if self.agent_pos in self.targets:
                reward = 2.0   # new target
            self.visited.add(self.agent_pos)
        else:
            reward = -1

        # Step count
        self.steps += 1

        # Terminal bonus
        terminated = True
        for (i,j) in self.targets:
            if (i,j) not in self.visited:
                terminated = False
                break
            
        truncated = (self.steps >= self.max_steps)
        if terminated:
            reward += 30.0

        return self._get_obs(), reward, terminated, truncated, {}

    def render(self, mode="human"):
        disp = np.full((self.H, self.W), '.', dtype=str)
        for (i,j) in self.targets:
            disp[i,j] = 'T'
        for (i,j) in zip(*np.where(self.grid == 1)):
            disp[i,j] = '#'
        ai, aj = self.agent_pos
        disp[ai, aj] = 'A'
        print("\n".join("".join(row) for row in disp))
        print("\n")

    def close(self):
        pass

In [22]:
gym.register(
    id="Coverage-v0",
    entry_point="coverage_env:CoverageEnv",
    max_episode_steps=200,
)


/Users/pedropertusi/Desktop/reinforcement-learning/Coverage-Path-Planning/env/lib/python3.12/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Coverage-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [23]:
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# instantiate a single env (you can wrap VecEnv for parallelism later)
env = CoverageEnv(seed=42)

# create the DQN model
model = PPO(
    policy="MlpPolicy",      # flatten your 5×8×8 obs into a 320‐dim vector
    env=env,             # or env directly if you prefer a single env
    learning_rate=1e-4,      # slightly lower than default 3e-4 for stability
    n_steps=2048,            # rollout length per update (≈10 episodes worth)
    batch_size=64,           # minibatch size for each epoch
    n_epochs=10,             # number of passes over the rollout buffer
    gamma=0.99,              # reward discount
    gae_lambda=0.95,         # GAE smoothing
    clip_range=0.2,          # PPO clipping parameter
    ent_coef=0.01,           # small entropy bonus to encourage exploration
    vf_coef=0.5,             # value function loss coefficient
    max_grad_norm=0.5,       # clip gradients
    verbose=1,
)

# train for 50k timesteps
model.learn(total_timesteps=100_000)

# save it
model.save("ppo_coverage")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 179      |
|    ep_rew_mean     | -139     |
| time/              |          |
|    fps             | 6506     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 185         |
|    ep_rew_mean          | -145        |
| time/                   |             |
|    fps                  | 3809        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008340899 |
|    clip_fraction        | 0.00483     |
|    clip_range           | 0.2         |
|    entropy_loss   

In [24]:
mean_reward, std_reward = evaluate_policy(
    model,
    env,
    n_eval_episodes=20,
    deterministic=True,
)
print(f"Mean reward: {mean_reward:.2f} ± {std_reward:.2f}")

Mean reward: 42.00 ± 0.00


/Users/pedropertusi/Desktop/reinforcement-learning/Coverage-Path-Planning/env/lib/python3.12/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [25]:
obs, _ = env.reset(seed=42)
for i in range(env.max_steps):
    action_arr, _ = model.predict(obs, deterministic=True)
    action = int(action_arr)       # unwrap numpy array
    print("step:", i, "action:", action)

    obs, reward, terminated, truncated, info = env.step(action)
    env.render()
    print()  # blank line between frames

    if terminated or truncated:
        print("Done!", "Terminated" if terminated else "Truncated")
        break

step: 0 action: 2
........
........
..TTT.#.
..TTT...
..####..
..#.#...
.##..A..
.#......



step: 1 action: 2
........
........
..TTT.#.
..TTT...
..####..
..#.#...
.##...A.
.#......



step: 2 action: 1
........
........
..TTT.#.
..TTT...
..####..
..#.#.A.
.##.....
.#......



step: 3 action: 1
........
........
..TTT.#.
..TTT...
..####A.
..#.#...
.##.....
.#......



step: 4 action: 1
........
........
..TTT.#.
..TTT.A.
..####..
..#.#...
.##.....
.#......



step: 5 action: 3
........
........
..TTT.#.
..TTTA..
..####..
..#.#...
.##.....
.#......



step: 6 action: 3
........
........
..TTT.#.
..TTA...
..####..
..#.#...
.##.....
.#......



step: 7 action: 3
........
........
..TTT.#.
..TAT...
..####..
..#.#...
.##.....
.#......



step: 8 action: 3
........
........
..TTT.#.
..ATT...
..####..
..#.#...
.##.....
.#......



step: 9 action: 1
........
........
..ATT.#.
..TTT...
..####..
..#.#...
.##.....
.#......



step: 10 action: 2
........
........
..TAT.#.
..TTT...
..####..
..#.#.